**Urban Mobility location services**

In [5]:
import requests
import folium
import flexpolyline

In [126]:
API_KEY = "YOUT_API_KEY"
TRAFFIC_FLOW_ENDPOINT = "https://traffic.ls.hereapi.com/traffic/6.3/flow.json"
TRANSIT_ENDPOINT = "https://transit.router.hereapi.com/v8/routes"
ROUTING_ENDPOINT = "https://router.hereapi.com/v8/routes"
WEATHER_ENDPOINT = "https://weather.ls.hereapi.com/weather/1.0/report.json"
ISOLINE_ENDPOINT = "https://isoline.router.hereapi.com/v8/isolines"
MAP_TILE_ENDPOINT = "https://1.base.maps.ls.hereapi.com/maptile/2.1/maptile/newest/normal.day/{z}/{x}/{y}/256/png8"

**Traffic Flow API**

In [15]:
def get_traffic(
    top_lat=39.8485715, 
    top_lng=-86.0969867, 
    bottom_lat=39.8358934,
    bottom_lng=-86.0757964
  ):
  traffic_params = {
      "apiKey": API_KEY,
      "bbox": f"{top_lat},{top_lng};{bottom_lat},{bottom_lng}",
      "responseattributes": "sh,fc",
  }

  response = requests.get(TRAFFIC_FLOW_ENDPOINT, params=traffic_params).json()

  return response

In [65]:
traffic_data = get_traffic()

In [92]:
traffic_map = folium.Map(
        location=[52.52923, 13.3881], 
        zoom_start=15,
        tiles="https://1.traffic.maps.ls.hereapi.com/traffic/6.0/tiles/{z}/{x}/{y}/256/png32?apiKey=" + API_KEY,
        attr='HERE'
    )
# traffic_map

**Public Transit API**

In [72]:
def get_transit_route(
    origin_lat=52.529798, 
    origin_lng=13.37903022, 
    destination_lat=52.53611, 
    destination_lng=13.407011
  ):
  transit_params = {
      "apiKey": API_KEY,
      "origin": f"{origin_lat},{origin_lng}",
      "destination": f"{destination_lat},{destination_lng}",
      "return": "polyline",
  }

  response = requests.get(TRANSIT_ENDPOINT, params=transit_params).json()

  return response

In [73]:
transit_data = get_transit_route()

In [78]:
route_sections = transit_data['routes'][0]['sections']

In [93]:
transit_map = folium.Map(
        location=[52.52923, 13.3881], 
        zoom_start=15,
        tiles="https://1.base.maps.ls.hereapi.com/maptile/2.1/maptile/newest/normal.day/{z}/{x}/{y}/256/png8?apiKey=" + API_KEY,
        attr='HERE'
    )

for route in route_sections:
  folium.PolyLine(flexpolyline.decode(route['polyline']), popup=f"Transport mode: {route['transport']['mode']}").add_to(transit_map)

# transit_map

**Isoline API** 

In [108]:
def get_isoline(
    origin_lat=52.52541, 
    origin_lng=13.38586,
    transport_type="pedestrian",  # car, bicycle, truck ...
    range_type='time', # time || distance
    value=500
  ):
  isoline_params = {
      "apiKey": API_KEY,
      "origin": f"{origin_lat},{origin_lng}",
      "transportMode": transport_type,
      "range[type]": range_type,
      "range[values]": value,
  }

  response = requests.get(ISOLINE_ENDPOINT, params=isoline_params).json()

  return response

In [113]:
isoline_data = get_isoline(origin_lat=52.52541, origin_lng=13.38586, transport_type="car")

In [115]:
isoline_map = folium.Map(
        location=[52.52541, 13.38586], 
        zoom_start=15,
        tiles="https://1.base.maps.ls.hereapi.com/maptile/2.1/maptile/newest/normal.day/{z}/{x}/{y}/256/png8?apiKey=" + API_KEY,
        attr='HERE'
    )

decoded_geometry = flexpolyline.decode(isoline_data['isolines'][0]['polygons'][0]['outer'])

folium.Polygon(decoded_geometry, fill="lightblue").add_to(isoline_map)

# isoline_map

**Destination Weather API**

In [124]:
def getWeather(lat, lon, apiKey):
  """
  https://developer.here.com/documentation/examples/rest/auto_weather/weather-observation-lat-long
  """

  params = {
      "product": "observation",
      "latitude": lat,
      "longitude": lon,
      "oneobservation": "true",
      "apiKey": apiKey,
  }

  response = requests.get(WEATHER_ENDPOINT, params=params).json()
  temperature = response['observations']['location'][0]['observation'][0]['temperature']
  
  return { "lat": lat, "lon": lon, "temp": temperature }

In [127]:
def create_marker(lat, lng):
  markerData = getWeather(lat, lng, API_KEY)
  marker = folium.Marker(location=[lat, lng], popup=f"{markerData['temp']}")
  return marker

weather_map = folium.Map(
        location=[55.7477, 37.62189], 
        zoom_start=10,
        tiles="https://1.base.maps.ls.hereapi.com/maptile/2.1/maptile/newest/normal.day/{z}/{x}/{y}/256/png8?apiKey=" + API_KEY,
        attr='HERE'
    )

coordinates = [
  (55.7477, 37.62189), 
  (55.7577, 37.62182), 
  (55.987477, 37.82189), 
  (55.22477, 37.72189), 
  (55.4477, 37.62189), 
  (55.3477, 37.14189)
]

for position in coordinates:
  marker = create_marker(position[0], position[1])
  marker.add_to(weather_map)


# weather_map